In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.linear_model import Lars,Ridge, Lasso, SGDClassifier,SGDRegressor,LogisticRegression,BayesianRidge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [4]:
path_to_file='/Users/ruonanding/Desktop/Liberty_Mutual/'
df_train = pd.read_csv(path_to_file+'train.csv')
df_test = pd.read_csv(path_to_file +'test.csv')

/Users/ruonanding/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ruonanding/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_train['target'].to_csv('ytrain.csv', sep = ',')

In [180]:
vvar=[]
for i in range(1,10):
    vvar.append('var'+str(i))

print "convert mixed columns to strings"
df_train[vvar] = df_train[vvar].applymap(str)
df_test[vvar] = df_test[vvar].applymap(str)

#assign dummy variables and remove the original column 
def make_dummies(df, variables):
    for variable in variables:
        dummies = pd.get_dummies(df[variable], prefix = variable)
        df = pd.concat([df, dummies], axis = 1)
        #df = df.drop(variable, axis = 1)
    return df

df_train = make_dummies(df_train, vvar)
df_test = make_dummies(df_test, vvar)

convert mixed columns to strings


In [158]:
nvar=[]
for i in range(10,18):
    nvar.append('var'+str(i))

#print "fill in NAs with mean of  columns to strings"
#df_train[geo] = df_train[geo].fillna(df_train[geo].mean())
#df_test[geo] = df_train[geo].fillna(df_train[geo].mean())

In [46]:
#now put all geo in one 
geo=[]
for i in range(1,38):
    geo.append('geodemVar'+str(i))

#fill the missing values with mean
df_train[geo] = df_train[geo].fillna(df_train[geo].mean())
df_test[geo] = df_train[geo].fillna(df_train[geo].mean())

In [184]:
crime=[]
for i in range(9):
    crime.append('crimeVar'+str(i+1))

#fill the missing values with mean
df_train[crime] = df_train[crime].fillna(df_train[crime].mean())
df_test[crime] = df_train[crime].fillna(df_train[crime].mean())


In [198]:
weather=[]
for i in range(1,237):
    weather.append('weatherVar'+str(i))

#fill the missing values with mean
df_train[weather] = df_train[weather].fillna(df_train[weather].mean())
df_test[weather] = df_train[weather].fillna(df_train[weather].mean())


In [61]:
##all the geo variables are decreased into 2 dimenstions pca
from sklearn.decomposition import PCA
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
geo_train_minmax = min_max_scaler.fit_transform(df_train[geo])
geo_test_minmax = min_max_scaler.fit_transform(df_test[geo])

pca = PCA()
geo_pca_train = pca.set_params(n_components = 2).fit_transform(geo_train_minmax)
geo_pca_test = pca.transform(geo_test_minmax)
#array([ 0.80294242,  0.06785476])

In [66]:
print "set binary labels"
df_train['target_class'] = (df_train.target>0).astype(int)

#class is the 1 or 0 whether to have a loss.
# loss is the previous target loss ratio. 
classes = df_train.target_class.values
loss = df_train.target.values

df_train = df_train.drop(['target', 'id', 'target_class'], axis = 1)
df_test = df_test.drop(['id'], axis = 1)

set binary labels


In [94]:
print "randomized lasso feature selector"
X_sp = np.asmatrix(df_train[weather])
from sklearn.linear_model import LogisticRegression
sel_log = LogisticRegression(C=0.005, random_state = 2016,penalty='l1').fit(X_sp, classes)

randomized lasso feature selector


In [132]:
from sklearn.feature_selection import SelectFromModel
linear_model = SelectFromModel(sel_log, prefit=True)

In [133]:
n_features = linear_model.transform(X_sp).shape[1]
n_features
## the regulated logistic regression gives us 51 features left

51

In [155]:
## Another L1 based feature selection.
from sklearn.svm import LinearSVC
lsvc = LinearSVC(C=0.005, penalty="l1", dual=False).fit(X_sp, classes)
svc_model = SelectFromModel(lsvc, prefit=True)
X_new = svc_model.transform(X_sp)
X_new.shape[1]
## SVC gives us 48 features. The lower the learning rate, the less features it will keep 

48

In [139]:
from sklearn.ensemble import ExtraTreesClassifier
tree_clf = ExtraTreesClassifier()
tree_clf = tree_clf.fit(X_sp, classes)
importance = tree_clf.feature_importances_    
indices = np.argsort(importance)[::-1]

In [144]:
tree_model = SelectFromModel(tree_clf, prefit=True)
X_new_tree = tree_model.transform(X_sp)
X_new_tree.shape[1]

78

In [156]:
weather_l1_train = svc_model.transform(X_sp)
weather_l1_test = svc_model.transform(np.asmatrix(df_test[weather]))

In [171]:
weather_trim=[]
for i in range(48):
    weather_trim.append('weahter_trim'+str(i+1))

In [186]:
## test using pca to do weather and crime
crime_train_minmax = min_max_scaler.fit_transform(df_train[crime])
crime_test_minmax = min_max_scaler.fit_transform(df_test[crime])

In [194]:
crimepca = PCA()
crime_pca= crimepca.set_params(n_components = 2).fit(crime_train_minmax)
crime_pca_train = crimepca.transform(crime_train_minmax)
crime_pca_test = crimepca.transform(crime_test_minmax)
#crime_pca.explained_variance_ratio_
#array([ 0.56717349,  0.21503951])

In [199]:
weather_train_minmax = min_max_scaler.fit_transform(df_train[weather])
weather_test_minmax = min_max_scaler.fit_transform(df_test[weather])

In [200]:
weather_pca= pca.set_params(n_components = 2).fit(weather_train_minmax)
weather_pca_train = weather_pca.transform(weather_train_minmax)
weather_pca_test = weather_pca.transform(weather_test_minmax)
weather_pca.explained_variance_ratio_
#array([ 0.27920987,  0.18910785])
#the reason i don't use pca on weather but rather L1 is that pca doesn't 
#explain much 

array([ 0.27920987,  0.18910785])

In [207]:
df_geo_pca_train = pd.DataFrame(geo_pca_train, columns=['pca_geo1', 'pca_geo2'])
df_geo_pca_test = pd.DataFrame(geo_pca_test, columns=['pca_geo1', 'pca_geo2'])

df_weather_train = pd.DataFrame(weather_l1_train, columns=[weather_trim])
df_weather_test = pd.DataFrame(weather_l1_test, columns=[weather_trim])

df_crime_pca_train = pd.DataFrame(crime_pca_train, columns=['pca_crime1', 'pca_crime2'])
df_crime_pca_test = pd.DataFrame(crime_pca_test, columns=['pca_crime1', 'pca_crime2'])

df_train[nvar]
df_test[nvar]

df_train[vvar]
df_test[vvar]

X_train = pd.concat([df_train[vvar],df_train[nvar],df_weather_train,
                     df_geo_pca_train,df_crime_pca_train], axis = 1)
X_test = pd.concat([df_test[vvar],df_test[nvar],df_weather_test,
                     df_geo_pca_test,df_crime_pca_test], axis = 1)

In [208]:
X_train.shape
### the training dataset has been reduced to 69 features. 

(452061, 69)

In [211]:
X_train.to_csv('xtrain.csv', sep=',')

In [212]:
X_test.to_csv('xtest.csv', sep=',')

In [1]:
X_train.shape

NameError: name 'X_train' is not defined

In [216]:
df_train['target'].to_csv('ytrain.csv', sep = ',')